In [1]:
import wptools
import pyspark
import pyspark.sql
from pyspark.sql import *
import os.path
from pyspark.sql.functions import desc

import findspark
findspark.init()

from pyspark.sql import dataframe
from pyspark.sql import functions as F

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

### Loading data 

In [2]:
DATA_DIR = '../' 
WIKIPEDIA_CONFLICTS_PARQUET = DATA_DIR + 'selectedConflict.parquet'

# loading the saved parquet files
wikipedia = spark.read.parquet(WIKIPEDIA_CONFLICTS_PARQUET)

In [3]:
wikipedia_temp = wikipedia.registerTempTable('wikipedia_temp')

categories_query = """
select DISTINCT categories
from wikipedia_temp"""

categories = spark.sql(categories_query)

In [ ]:
categories.show(10)

In [ ]:
wikipedia.where("categories like '%civilian attack%'").show(5) #to change to military conflict in script

In [ ]:
wikipedia.filter("ns = '0'") \
            .select("revision.text._VALUE") \
            .filter("_VALUE like '%{{%Infobox civilian attack'") \
            .show(2)

In [ ]:
wikipedia.where("title = 'Tulsa race riot'").show()

In [ ]:
wikipedia.where("title = 'Tulsa race riot'") \
        .select("revision.text._VALUE") \
        .filter("_VALUE like '%{{Infobox civilian attack%'").collect()[0]

### UNICORN ON THE GOOO

In [4]:
bubu_page = wptools.page('Tulsa race riot')
#dir(page.get_parse())
bubu_page.get_parse()

en.wikipedia.org (parse) Tulsa race riot
en.wikipedia.org (imageinfo) File:TulsaRaceRiot-1921.png
Tulsa race riot (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:TulsaRace...
  infobox: <dict(13)> title, partof, image, image_upright, caption...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:T...
  pageid: 30785
  parsetree: <str(92653)> <root><template><title>Use mdy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Tulsa race riot
  wikibase: Q1824714
  wikidata_url: https://www.wikidata.org/wiki/Q1824714
  wikitext: <str(75336)> {{Use mdy dates|date=March 2016}}{{Infobo...
}


In [5]:
bubu_page.get_more()

en.wikipedia.org (querymore) 30785
Tulsa race riot (en) data
{
  backlinks: <list(412)> {'pageid': 15080, 'ns': 0, 'title': 'Indi...
  categories: <list(20)> Category:1921 crimes, Category:1921 in Ok...
  contributors: 955
  files: <list(16)> File:All That Was Left of His Home after the T...
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:TulsaRace...
  infobox: <dict(13)> title, partof, image, image_upright, caption...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:T...
  languages: <list(11)> {'lang': 'da', 'title': 'Raceoptøjerne i T...
  pageid: 30785
  parsetree: <str(92653)> <root><template><title>Use mdy dates</ti...
  requests: <list(3)> parse, imageinfo, querymore
  title: Tulsa race riot
  views: 1,916
  wikibase: Q1824714
  wikidata_url: https://www.wikidata.org/wiki/Q1824714
  wikitext: <str(75336)> {{Use mdy dates|date=March 2016}}{{Infobo...
}


In [6]:
bubu_page.data['infobox']

{'title': 'Tulsa race riot',
 'partof': '[[racism in the United States]]',
 'image': 'TulsaRaceRiot-1921.png',
 'image_upright': '1.3',
 'caption': 'Buildings burning during the Tulsa race riot',
 'location': '[[Greenwood, Tulsa]], Oklahoma, U.S.',
 'date': 'May 31 – June 1, 1921',
 'coordinates': '{{coord |36.1594 |N |95.9864 |W |region:US-OK_type:event |display|=|inline,title|format|=|dms}}',
 'fatalities': '36 (1921 findings)<br />Total unknown according to Red Cross <br>100–300 (2001 Oklahoma Commission Report)',
 'injuries': 'Over 800',
 'target': 'Black citizens',
 'perps': 'White mob, the police, the [[United States National Guard]] {{rp|193, 196}}',
 'weapons': 'Guns, incendiary devices, explosives, airplanes {{rp|196}}'}

In [7]:
print(bubu_page.data['views'])
print(bubu_page.data['infobox']['location'])
print(bubu_page.data['infobox']['date'])
print(bubu_page.data['infobox']['fatalities'])
print(bubu_page.data['infobox']['injuries'])

1916
[[Greenwood, Tulsa]], Oklahoma, U.S.
May 31 – June 1, 1921
36 (1921 findings)<br />Total unknown according to Red Cross <br>100–300 (2001 Oklahoma Commission Report)
Over 800


---

### Infobox per category ?

### Infobox `civilian attack`

In [8]:
#to change to military conflict in script
infobox = 'civilian attack'
# find all pages that have category civilian attack
wiki_civil_attack = wikipedia.where("categories like '%{}%'".format(infobox)) 
# show file schema
wiki_civil_attack.printSchema()

root
 |-- id: long (nullable = true)
 |-- ns: long (nullable = true)
 |-- restrictions: string (nullable = true)
 |-- revision: struct (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |-- contributor: struct (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- ip: string (nullable = true)
 |    |    |-- username: string (nullable = true)
 |    |-- format: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- minor: string (nullable = true)
 |    |-- model: string (nullable = true)
 |    |-- parentid: long (nullable = true)
 |    |-- sha1: string (nullable = true)
 |    |-- text: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _space: string (nullable = true)
 |    |-- timestamp: string (nullable = true)
 |-- title: string (nullable = true)
 |-- article_lenght: integer (nullable = true)
 |-- c

# TRIAL ===================================
def extract_infobox_civilian_attack(entity):
    # get page
    page = wptools.page(entity.title)
    page.get_parse()
    page.get_more()
    # extract relevant information
    views_ = page.data['views']
    location_ = page.data['infobox']['location']
    date_ = page.data['infobox']['date']
    fatalities_ = page.data['infobox']['fatalities']
    #injuries_ = page.data['infobox']['injuries']
    
    
    return Row(id=entity.id, title=entity.title, location=location_, views=views_, date=date_, 
                fatalities=fatalities_ )# , injuries=injuries_ 


def extract_infobox_civil_conflict(entity):
    # get page
    page = wptools.page(entity.title)
    page.get_parse()
    page.get_more()
    # extract relevant information
    views_ = page.data['views']
    location_ = page.data['infobox']['place']
    date_ = page.data['infobox']['date']
    fatalities_ = page.data['infobox']['fatalities']
    #injuries_ = page.data['infobox']['injuries']
    casualities1_ = page.data['infobox']['casualties1']
    casualities2_ = page.data['infobox']['casualties2']
    leadfigures1_ = page.data['infobox']['leadfigures1'] 
    leadfigures2_ = page.data['infobox']['leadfigures2']
    
    
    return Row(id=entity.id, title=entity.title, location=location_, views=views_, date=date_, 
                fatalities=fatalities_, casualities1=casualities1_, casualities2=casualities2_,
              leadfigures1=leadfigures1_, leadfigures2=leadfigures2_)# , injuries=injuries_ 

def extract_infobox_military_conflict(entity):
    # get page
    page = wptools.page(entity.title)
    page.get_parse()
    page.get_more()
    # extract relevant information
    views_ = page.data['views']
    location_ = page.data['infobox']['place']
    date_ = page.data['infobox']['date']
    fatalities_ = page.data['infobox']['fatalities']
    #injuries_ = page.data['infobox']['injuries']
    casualities1_ = page.data['infobox']['casualties1']
    casualities2_ = page.data['infobox']['casualties2']
    combatant1_ = page.data['infobox']['combatant1'] 
    combatant2_ = page.data['infobox']['combatant2']
    
    
    return Row(id=entity.id, title=entity.title, location=location_, views=views_, date=date_, 
                fatalities=fatalities_, casualities1=casualities1_, casualities2=casualities2_,
              combatant1=combatant1_, combatant2=combatant2_)# , injuries=injuries_ 
              
              
# THE END OF TRIAL

In [14]:
# GET INFO FROM INFOBOX + VIEWS for different categories
def get_infobox_civilian_attack(entity):
    # get page
    page = wptools.page(entity.title)
    page.get_parse()
    page.get_more()
    # extract relevant information and put in dictionary
    info = {'views': None, 'location': None, 
            'date': None, 'fatalities': None, 'injuries': None }
    try:
        info['views'] = page.data['views']
    except KeyError:
        info['views'] = None
    
    for ele in list(info.keys())[1:]:
        try:
            info[ele] = page.data['infobox'][ele]
        except KeyError:
            pass
    
    return Row(id=entity.id, title=entity.title, location=info['location'], views=info['views'], date=info['date'], 
                fatalities=info['fatalities'], injuries=info['injuries'])

def get_infobox_civil_conflict(entity):
    # get page
    page = wptools.page(entity.title)
    page.get_parse()
    page.get_more()
    # extract relevant information and put in dictionary
    info = {'views': None, 'place': None, 'injuries': None
            'date': None, 'fatalities': None, 'casualties1': None, 'casualties2': None,
            'leadfigures1': None, 'leadfigures2': None} 
    try:
        info['views'] = page.data['views']
    except KeyError:
        info['views'] = None
    
    for ele in list(info.keys())[1:]:
        try:
            info[ele] = page.data['infobox'][ele]
        except KeyError:
            pass
    
    return Row(id=entity.id, title=entity.title, location=info['place'], views=info['views'], date=info['date'], 
               fatalities=info['fatalities'], casualties1=info['casualties1'], casualties2=info['casualties2'],
               injuries=info['injuries'], leadfigures1=info['leadfigures1'], leadfigures2=info['leadfigures2'])

def get_infobox_military_conflict(entity):
    # get page
    page = wptools.page(entity.title)
    page.get_parse()
    page.get_more()
    # extract relevant information and put in dictionary
    info = {'views': None, 'place': None, 
            'date': None, 'casualties1': None, 'casualties2': None,
            'combatant1': None, 'combatant2': None, 'status': None} 
    try:
        info['views'] = page.data['views']
    except KeyError:
        info['views'] = None
    
    for ele in list(info.keys())[1:]:
        try:
            info[ele] = page.data['infobox'][ele]
        except KeyError:
            pass
    
    return Row(id=entity.id, title=entity.title, location=info['place'], views=info['views'], date=info['date'], 
               casualties1=info['casualties1'], casualties2=info['casualties2'], combatant1=info['combatant1'], 
               combatant2=info['combatant2'], status=info['status'])

In [15]:
wiki_civil_attack_RDD = sqlContext.createDataFrame(wiki_civil_attack.rdd.map(get_infobox_civilian_attack))



In [16]:
wiki_civil_attack_RDD.take(2)

[Row(date=None, fatalities='168 confirmed + 1 possible additional fatality', id=22467, injuries='680+', location='[[Alfred P. Murrah Federal Building]]<br />[[Oklahoma City]], [[Oklahoma]], U.S.', title='Oklahoma City bombing', views=4319),
 Row(date='16 March 1968', fatalities='347 according to the United States Army (not including My Khe killings), others estimate more than 400 killed and injuries are unknown, Vietnamese government lists 504 killed in total from both My Lai and My Khe', id=68292, injuries=None, location='[[Son My|Sơn Mỹ]] (village), [[Sơn Tịnh District]], [[South Vietnam]]', title='My Lai Massacre', views=2251)]